In [1]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

In [2]:
import astro_time
import sgp4
import sensor

In [5]:
import public_astrostandards as PA
# PA.get_versions()

In [12]:
obs = pd.read_json('/home/woodkn1/Downloads/19548.json.gz').sort_values(by='obTime').reset_index(drop=True)
obs['obTime_dt'] = pd.to_datetime( obs['obTime'] )

In [43]:
# setup our canonical date format
date_df = astro_time.convert_times( obs['obTime_dt'], PA )
# append that to the obs frame
obs_df = pd.concat( (obs.reset_index(drop=True),date_df.reset_index(drop=True)), axis=1 )

# get the sensor position in TEME ECI
def sensorECI( O, harness ):
    lla = (harness.ctypes.c_double * 3)( O['senlat'], O['senlon'], O['senalt'] )
    eci = (harness.ctypes.c_double * 3)( )
    harness.AstroFuncDll.LLHToXYZTime( O['ds50_utc'], lla, eci )
    return list( eci )

sensor_df = date_df.copy()
sensor_df['teme_p'] = obs_df.apply( lambda X: sensorECI(X,PA), axis=1 )
sensor_df['lon'] = obs['senlon']
sensor_df['lat'] = obs['senlat']

In [45]:
def rotateTEME( O , harness ):
    newRA  = (harness.ctypes.c_double)()
    newDec = (harness.ctypes.c_double)()
    PA.AstroFuncDll.RotRADec_EqnxToDate( 106,
                                        2,
                                        O['ds50_utc'],
                                        O['ra'],
                                        O['declination'],
                                        newRA,
                                        newDec )
    return ( np.float64(newRA),np.float64(newDec) )
                                    
                                        
tmp = obs_df.apply( lambda X : rotateTEME( X, PA ) , axis=1)
obs_df['teme_ra']  = [ X[0] for X in tmp ]
obs_df['teme_dec'] = [ X[1] for X in tmp ]
obs_df[['ra','teme_ra']]

,ra,teme_ra
0,324.147131,324.477809
1,324.196245,324.526905
2,324.219747,324.550398
3,324.244990,324.575631
4,324.250537,324.581101
...,...,...
1060,320.564392,320.908839
1061,320.588870,320.933300
1062,320.613251,320.957665
1063,320.662031,321.006412


In [ ]:
# setup the sensor frame

In [47]:
tle_dates_df = date_df.copy()
eph = sgp4.propTLE_df( tle_dates_df, 
                '1 19548U 88091B   25281.05493527 -.00000297  00000+0  00000+0 0  9993',
                '2 19548  12.7961 342.6596 0038175 340.8908  24.1736  1.00278194122860',
                PA)
test_looks = sensor.compute_looks( sensor_df, tle_dates_df, PA )

In [50]:
test_looks['XA_TOPO_RA'] - obs_df['teme_ra']

0       0.023260
1       0.023110
2       0.024007
3       0.023262
4       0.023454
          ...   
1060    0.001932
1061    0.001982
1062    0.002044
1063    0.002102
1064    0.001912
Length: 1065, dtype: float64